# Joins 

SQL supports joining different tables together like pandas.

Or, rather, pandas copied SQL's capabilities in this area.

In [2]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data/demo.db3')
# Check available tables
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con=conn)

,name
0,rch
1,hru
2,sub
3,sed
4,watershed_daily
5,watershed_monthly
6,watershed_yearly
7,channel_dimension
8,hru_info
9,sub_info


the rch table contains time series data with year and month for each river reach. Therefore, it is natural to calculate some seasonal statistics. 

In [3]:
pd.read_sql("SELECT * From rch LIMIT 3", con=conn)

,RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,EVAPcms,TLOSScms,SED_INtons,SED_OUTtons,SEDCONCmg_kg,...,BED_PSTmg,BACTP_OUTct,BACTLP_OUTct,CMETAL_1kg,CMETAL_2kg,CMETAL_3kg,TOT_Nkg,TOT_Pkg,NO3ConcMg_l,WTMPdegc
0,1,1981,1,146.343765,146.252487,0.091281,0.0,2.332046e-07,6.161946e+04,1.553719e+02,...,0.0,0.0,0.0,0.0,0.0,0.0,806.015747,0.000000,0.0,0.0
1,2,1981,1,96.225693,96.182854,0.042821,0.0,1.642676e-07,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,442.463013,0.000000,0.0,0.0
2,3,1981,1,11.952719,11.861368,0.091352,0.0,2.032582e-07,2.032582e-07,6.595061e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,110.399124,0.009118,0.0,0.0


In [4]:
pd.read_sql("SELECT * From sub LIMIT 3", con=conn)

,SUB,YR,MO,PRECIPmm,SNOMELTmm,PETmm,ETmm,SWmm,PERCmm,SURQmm,...,SEDPkg_ha,LAT_Q_mm,LATNO3kg_h,GWNO3kg_ha,CHOLAmic_L,CBODU_mg_L,DOXQ_mg_L,TNO3kg_ha,QTILEmm,TVAPkg_ha
0,1,1981,1,35.601990,0.0,3.720744,0.249642,10.798594,0.0,0.000024,...,4.759284e-08,0.007812,0.000000,1.562287e-07,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1981,1,108.606071,0.0,3.450408,0.457206,56.325005,0.0,0.000017,...,3.447771e-08,0.033931,0.000016,3.828798e-04,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1981,1,149.308365,0.0,10.566324,6.027106,71.002098,0.0,0.000024,...,3.778229e-08,0.007915,0.000004,7.582882e-05,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df = pd.read_sql(
"""
SELECT 
    RCH, 
    rch.YR, 
    rch.MO, 
    FLOW_INcms, 
    FLOW_OUTcms, 
    PRECIPmm, 
    PETmm
FROM rch 
INNER JOIN sub
    ON rch.RCH = sub.SUB 
    and rch.YR=sub.YR and rch.MO=sub.MO
""", con=conn)

print(len(df))
df.head()

8280


,RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
0,1,1981,1,146.343765,146.252487,35.601990,3.720744
1,2,1981,1,96.225693,96.182854,108.606071,3.450408
2,3,1981,1,11.952719,11.861368,149.308365,10.566324
3,4,1981,1,49.486492,49.406513,108.606049,10.674994
4,5,1981,1,274.066803,272.106018,201.311279,27.179243


Different join types are supported in the `JOIN` statement (`LEFT JOIN`, `OUTER JOIN`, etc.), and `WHERE` statements.

The `WHERE` statements need to specify which table we're restricting:

In [13]:
df = pd.read_sql(
"""
SELECT 
    RCH, 
    rch.YR, 
    rch.MO, 
    FLOW_INcms, 
    FLOW_OUTcms, 
    PRECIPmm, 
    PETmm
FROM rch 
LEFT JOIN sub
    ON rch.RCH = sub.SUB 
    and rch.YR=sub.YR and rch.MO=sub.MO
WHERE
    YR > 1984
""", con=conn)

print(len(df))
df.head()

DatabaseError: Execution failed on sql '
SELECT 
    RCH, 
    rch.YR, 
    rch.MO, 
    FLOW_INcms, 
    FLOW_OUTcms, 
    PRECIPmm, 
    PETmm
FROM rch 
LEFT JOIN sub
    ON rch.RCH = sub.SUB 
    and rch.YR=sub.YR and rch.MO=sub.MO
WHERE
    YR > 1984
': ambiguous column name: YR

Since two tables have the `YR` field, we need to specify which we're subsetting.

Here we'll subset the left side and do a left join, yielding a smaller result:

In [16]:
df = pd.read_sql(
"""
SELECT 
    RCH, 
    rch.YR, 
    rch.MO, 
    FLOW_INcms, 
    FLOW_OUTcms, 
    PRECIPmm, 
    PETmm
FROM rch 
LEFT JOIN sub
    ON rch.RCH = sub.SUB 
    and rch.YR=sub.YR and rch.MO=sub.MO
WHERE
    rch.YR > 1984
""", con=conn)

print(len(df))
df.head()

7176


,RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
0,1,1985,1,65.014122,64.851318,46.502602,6.640418
1,2,1985,1,44.200371,44.159149,103.605797,3.325713
2,3,1985,1,2.623852,2.441326,119.506691,21.123943
3,4,1985,1,20.668079,20.552505,103.605797,15.438934
4,5,1985,1,354.287933,348.826935,146.608200,75.672722


### Calculate Seasonal Runoff

There are two key steps: 
>(1) use the CASE and Subquery to convert months to named seasons;<br>
>(2) calculate seasonal mean with aggregate functions on groups.

In addition, we also use another filter keyword of ***BETWEEN*** to span months into seasons.

In [ ]:
df = pd.read_sql(
"""
SELECT 
    RCH, 
    Quarter, 
    AVG(FLOW_OUTcms) as Runoff
--- This is a subquery
FROM (
    SELECT RCH, YR, 
    CASE 
        WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
        WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
        WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
        ELSE 'DJF' 
    END Quarter,
    FLOW_OUTcms
    from rch
)
GROUP BY RCH, Quarter
""", con=conn)

print(len(df))
df.head()